In [ ]:
# ! lamin init --storage ./testdb --name test --modules bionty
# ! lamin connect anonymous/testdb

In [2]:
from scdataloader.utils import _adding_scbasecamp_genes, populate_my_ontology
import os.path
import urllib.request
import torch

%load_ext autoreload
%autoreload 2

→ connected lamindb: jkobject/scprint2


In [ ]:
populate_my_ontology(
    organisms_clade=["vertebrates"],
    sex=["PATO:0000384", "PATO:0000383"],
    organisms=["NCBITaxon:10090", "NCBITaxon:9606"],
    # celltypes=None,
    # ethnicities=None,
    # assays=None,
    # tissues=None,
    # diseases=None,
    # dev_stages=None,
)
_adding_scbasecamp_genes()

In [ ]:
LOC = "../../models/"  # "../../../"
ckpt_path = os.path.join(LOC, "18hebyht-final-small.ckpt")
if not os.path.exists(ckpt_path):
    url = (
        "https://huggingface.co/jkobject/scPRINT/resolve/main/18hebyht-final-small.ckpt"
    )
    urllib.request.urlretrieve(url, ckpt_path)

# OPTIONAL: solving potential issues

some models have additional elements that are not used anymore and can cause an
issue when loading them on specific version of pytorch lightning and scPRINT or
scPRINT2


In [ ]:
m = torch.load(
    model_checkpoint_file, map_location=torch.device("cpu"), weights_only=False
)

In [ ]:
m["hyper_parameters"]["label_decoders"] = {
    k: {u: j if j is not None else "None" for u, j in v.items()}
    for k, v in m["hyper_parameters"]["label_decoders"].items()
}
rn = {
    "expr_encoder.encoder.2.weight": "expr_encoder.encoder.1.weight",
    "expr_encoder.encoder.2.bias": "expr_encoder.encoder.1.bias",
    "expr_encoder.encoder.6.weight": "expr_encoder.encoder.5.weight",
    "expr_encoder.encoder.6.bias": "expr_encoder.encoder.5.bias",
    "expr_decoder.fc.2.weight": "expr_decoder.fc.1.weight",
    "expr_decoder.fc.2.bias": "expr_decoder.fc.1.bias",
    "expr_decoder.fc.6.weight": "expr_decoder.fc.5.weight",
    "expr_decoder.fc.6.bias": "expr_decoder.fc.5.bias",
}
m["state_dict"] = {k if k not in rn else rn[k]: v for k, v in m["state_dict"].items()}

m["hyper_parameters"].pop("checkpointing")
m["hyper_parameters"].pop("residual_in_fp32")
m["hyper_parameters"].pop("fused_dropout_add_ln")
m["hyper_parameters"].pop("checkpointing")
m["hyper_parameters"].pop("fused_mlp")
m["hyper_parameters"].pop("fused_bias_fc")
m["hyper_parameters"].pop("drop_path_rate")
m["hyper_parameters"].pop("class_compression")
m["hyper_parameters"].pop("depth_atinput")
m["hyper_parameters"].pop("cell_transformer_layers")
m["hyper_parameters"].pop("residual_in_fp32")

In [ ]:
torch.save(m, model_checkpoint_file)